In [2]:
from langchain_text_splitters import RecursiveCharacterSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

ImportError: cannot import name 'RecursiveCharacterSplitter' from 'langchain_text_splitters' (C:\Users\Roger\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_text_splitters\__init__.py)

In [3]:
import os

In [4]:
os.environ["OPEN_API_KEY"] = ""

In [5]:
#Loading models (Embeddings + LLM)

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens= 200)


NameError: name 'OpenAIEmbeddings' is not defined

In [ ]:
#Loading PDF

pdf_link = ""

loader = pyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [ ]:
#Split into chunks

text_splitter = RecursiveCharacterSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
    chunks = text_splitter.split_documents(pages)
)

In [ ]:
#Save to VectorDB (Chroma)

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
db.persist()

In [ ]:
#Loading DB

vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

#Loading Retriever

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

#Build prompt chain

chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))["output_text"]

    return answer

In [ ]:
question = input("User: ")
answer = ask(question)
print("Answer: ", answer)